In [2]:

 
import matplotlib.pylab as plt
from PIL import Image
from io import BytesIO
import gzip
import binascii
import zlib

def extract_packets():
    allpackets = []
    with open('htmlhexdump.txt', 'r') as file:
        packet = ''
        for line in file:
            temp = line.split('  ')
            if len(temp) >= 3:
                if(temp[0].startswith('0000')):
                    packet = ''
                packet += ''.join(temp[1].split(' '))
            if len(temp) == 1:
                allpackets.append(packet)
    return allpackets

def parse_packet(data):
    hexdata = bytes.fromhex(data)
    
    #/* handling frame header */
    ethernet_header_len = 14
    
    #/* handling IP headers */
    ipheader_offset = ethernet_header_len
    ipheader = hexdata[ipheader_offset: ipheader_offset+20]
    ipheader_len = (ipheader[0] & 0x0F)*4
    
    #/* handling TCP segment header */
    tcpheader_offset = ethernet_header_len + ipheader_len
    tcpheader = hexdata[tcpheader_offset: tcpheader_offset+20]
    tcplen = ((tcpheader[12] >> 4) & 0x0F) * 4
    
    httpheader_offset = tcpheader_offset + tcplen
    httpdata = hexdata[httpheader_offset:]
    return httpdata

'''
use this to log packets ...
tshark -r web.pcap -Y "frame.number == _frame_num" -x > output.txt
for each frame number

ensure that file has a newline at the end
'''
pkts = extract_packets()
httpdata = b''
for pkt in pkts:
    httpdata += parse_packet(pkt)

	
offset = httpdata.find(b'\r\n\r\n') + 4
mydata = httpdata[offset:]



#/* handling image data */
#gif = Image.open(BytesIO(mydata))
#plt.imshow(gif)
#plt.show()

#/* handling text data */
comp_data = len(mydata)
print("Reassembled compressed data:", comp_data)

uncomp_data = len(gzip.decompress(mydata).decode('utf-8'))

print("Uncompressed data size:", uncomp_data)

print("Compression Ratio:", uncomp_data/comp_data)





Reassembled compressed data: 31249
Uncompressed data size: 153715
Compression Ratio: 4.919037409197094
